In [ ]:
from IPython.display import display

import torch
from torch import nn
from torch.nn import functional

torch.set_printoptions(sci_mode=False)
torch.set_default_tensor_type('torch.FloatTensor')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class Net(nn.Module):
    def __init__(self, n_channels):
        super(Net, self).__init__()
        
        self.n_channels = n_channels

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=self.n_channels, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(n_channels, n_channels*2, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(n_channels*2, n_channels*4, kernel_size=3, stride=1, padding=1)

        self.fc1 = nn.Linear(self.n_channels*4*8*8, self.n_channels*4) # Entrada la salida de n_dims de foward
        self.fc2 = nn.Linear(self.n_channels*4, 6)

    def forward(self, x):
        # Empieza 3x64x64
        x = self.conv1(x) # n_channels x 64 x 64
        x = functional.relu(functional.max_pool2d(x, 2)) # max_pool divide dimension de imagen | n_chanels x 32 x 32
        
        x = self.conv2(x) # n_channels*2 x 32 x 32
        x = functional.relu(functional.max_pool2d(x, 2)) # n_chanels*2 x 16 x 16
        
        x = self.conv3(x) # n_channels*4 x 16 x 16
        x = functional.relu(functional.max_pool2d(x, 2)) # n_chanels*4 x 8 x 8

        # Flatten, aplanar datos a una dimension
        x = x.view(-1, self.n_channels*4*8*8)
        
        # Fully Connected (FC)
        x = self.fc1(x) # out=n_cannels*4
        x = functional.relu(x) 
        x = self.fc2(x) # Salida cantidad de clases a predecir

        # log_SoftMax
        x = functional.log_softmax(x, dim=1) # Salida en forma de probabilidad

        return x

In [ ]:
import sys
sys.path.append('..')

from plot_helpers import imshow

import os 
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [ ]:
class SignsDataset(Dataset):
    def __init__(self, base_dir, split_name='train', transform=None):
        path = os.path.join(base_dir, '{}_signs'.format(split_name))
        files = os.listdir(path)

        self.filenames = [ os.path.join(path, file) for file in files if file.endswith('.jpg') ]
        self.targets = [ int(file[0]) for file in files ]
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
        image = Image.open(self.filenames[index])
        image = self.transform(image) if self.transform else image
        return image, self.targets[index]



In [ ]:
signs = SignsDataset('../datasets/64x64_SIGNS')

display(len(signs))
display(signs[0][1])
display(signs[0][0])

In [ ]:
from torchvision import transforms

train_set = SignsDataset('../datasets/64x64_SIGNS', split_name='train', transform=transforms.ToTensor())
dataloader = DataLoader(train_set, batch_size=32, shuffle=True)

In [ ]:
from torchvision.utils import make_grid

for inputs, targets in dataloader:
    print(targets)
    out = make_grid(inputs)
    imshow(out)
    break

In [ ]:
from torch import optim


In [ ]:
net = Net(32).to(device)

loss_fn = nn.NLLLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9)

In [ ]:
class RunningMetric():
    def __init__(self):
        self.summatory = 0
        self.n_data = 0

    def update(self, val, size):
        self.summatory += val
        self.n_data += size 

    def __call__(self):
        return self.summatory/float(self.n_data)

In [ ]:
n_epochs = 100

for epoch in range(n_epochs):
    print(f'Epoch {epoch+1}/{n_epochs}')
    print('_'*10, '\n')

    running_loss = RunningMetric() # perdida
    running_acc = RunningMetric() # precision

    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        outputs = net(inputs)
        _, preds = torch.max(outputs, 1)
        loss = loss_fn(outputs, targets)

        loss.backward() # magias: gradientes calculados automaticamente
        optimizer.step() # magia2: actualiza las pesos

        batch_size = inputs.size()[0]
        running_loss.update(loss.item()*batch_size, batch_size)
        running_acc.update(torch.sum(preds == targets).float(), batch_size)

    print( 'Loss:  {:.4f}      Acc: {:.3%}'.format(running_loss(), running_acc()) )
